<a href="https://colab.research.google.com/github/DAlkemade/bert-for-fever/blob/master/L101_evaluate_sentence_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from google.colab import drive
import json
import pickle
import numpy as np
import os
import copy
import pprint
import random
import sqlite3
import pandas as pd

In [0]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
docs_input_file = '/content/drive/My Drive/Overig/dev_complete.sentences_id=k5t.p5.s5.jsonl'
N = 5
pp = pprint.PrettyPrinter(indent=4)
PERMUTATION = False
WORK_DIR = '/content/drive/My Drive/Overig/'

In [0]:
with open(docs_input_file, "r") as in_file:
    instances_input = []
    for line in in_file:
        instances_input.append(json.loads(line))
print(len(instances_input))
baseline_prediction = copy.deepcopy(instances_input)

for instance in instances_input:
    instance.pop('predicted_sentences', None) # drop all predicted sentences, since that's what we're doing

19998


In [0]:
with open('/content/drive/My Drive/Overig/sentence_evidence_from191219145714features_include_title=True_from_dev_from_baseline_docs_complete_fixed.pkl', 'rb') as f:
    evidence_titles = pickle.load(f)
with open('/content/drive/My Drive/Overig/sentence_evidence_all_from191219145714features_include_title=True_from_dev_from_baseline_docs_complete_fixed.pkl', 'rb') as f:
    evidence_titles_all_scores = pickle.load(f)
with open('/content/drive/My Drive/Overig/sentence_evidence_fromfeatures_dev_from_baseline_docs_complete_fixed.pkl', 'rb') as f:
    evidence_no_titles = pickle.load(f)
with open('/content/drive/My Drive/Overig/sentence_evidence_all_fromfeatures_dev_from_baseline_docs_complete_fixed.pkl', 'rb') as f:
    evidence_no_titles_all_scores = pickle.load(f)
with open('/content/drive/My Drive/Overig/sentence_evidence_from200110134032features_include_title=False_from_dev_sentences_from_bert_doc_selector.pkl', 'rb') as f:
    evidence_bert = pickle.load(f)
with open('/content/drive/My Drive/Overig/sentence_evidence_all_from200110134032features_include_title=False_from_dev_sentences_from_bert_doc_selector.pkl', 'rb') as f:
    evidence_all_bert = pickle.load(f)

In [0]:
# mean_number_evidence_predictions_per_claim = np.mean([len(value) for value in evidence.values()])
# mean_number_evidence_predictions_per_claim

In [0]:
import heapq

def save_similar(evidence, n_max=N, save=True):
    instances = copy.deepcopy(instances_input)
    for instance in instances:
        id = instance['id']
        if id in evidence.keys(): # so if verifiable
            predicted_sentences = evidence[id]
            # print(predicted_sentences)
            predicted_sentences_sorted = sorted(predicted_sentences, key=lambda tup: tup[0])
            # print(predicted_sentences_sorted)

            n= min(n_max,len(predicted_sentences_sorted))
            prediction = [[l[1], l[2]] for l in predicted_sentences_sorted[-n:]]
        else: # so if not verifiable
            prediction = [] # maybe random article instead?
        instance['predicted_sentences'] = prediction
    
    if save:
        with open(os.path.join(WORK_DIR, f'bert_predictions_sentences.jsonl'), "w+") as out_file:
            for instance in instances:
                out_file.write(json.dumps(instance) + "\n")
        print(f"Saved file")  
    return instances          


In [0]:
predictions_titles_high_precision = save_similar(evidence_titles, save=False)
predictions_titles_high_recall = save_similar(evidence_titles_all_scores, save=False)
predictions_no_titles_high_precision = save_similar(evidence_no_titles, save=False)
predictions_no_titles_high_recall = save_similar(evidence_no_titles_all_scores, save=False)
predictions_bert_docs = save_similar(evidence_bert, save=True)
predictions_all_bert_docs = save_similar(evidence_all_bert, save=False)

Saved file


In [0]:
def evaluate(instances_tmp):
    results = []
    prediction_lengths = []
    
    avoidable_mistakes = []
    for instance in instances_tmp:
        id = instance['id']
        if instance['verifiable'] == 'VERIFIABLE': # so if verifiable

            predicted = instance['predicted_sentences']
            predicted_pages = instance['predicted_pages']
            prediction_lengths.append(len(predicted))
            # print(all_evi)
            # print(f'{predicted}\n')
            found = 0
            for evidence_group in instance["evidence"]:
                evidence = [[e[2], e[3]] for e in evidence_group]
                if all([item in predicted for item in evidence]):
                    # We only want to score complete groups of evidence. Incomplete groups are worthless.
                    found = 1

            if found == 0:
                for evidence_group in instance["evidence"]:
                    evidence = [[e[2], e[3]] for e in evidence_group]
                    for item in evidence:
                        if not item in predicted and item[0] in predicted_pages:
                            avoidable_mistakes.append([id, instance['claim'], item[0], item[1], instance['label'], predicted])

               
                            
            results.append(found)
                
    print(f'Fully supported fraction {np.mean(results)}')
    print(f'Number of checked claims: {len(results)}')
    print(f'Average prediction length: {np.mean(prediction_lengths)}')
    return results, avoidable_mistakes

In [0]:
recall_arrays = {}
print("Baseline")
recall_arrays['baseline'], errors = evaluate(baseline_prediction)
print("Titles 1")
recall_arrays['titles+force5'], _ = evaluate(predictions_titles_high_recall)
print("Titles 2")
recall_arrays['titles+noforce'], _ = evaluate(predictions_titles_high_precision)
print("No titles 1")
recall_arrays['no_titles+force5'], _ = evaluate(predictions_no_titles_high_recall)
print("No titles 2")
recall_arrays['no_titles+noforce'], _ = evaluate(predictions_no_titles_high_precision)
print("Bert")
recall_arrays['bertdocs'], _ = evaluate(predictions_bert_docs)
print("Bert all")
recall_arrays['bertdocs_all'], _ = evaluate(predictions_all_bert_docs)

In [0]:
# from L90 project
R = 5000  # Number of permutations


def single_permute(results_a, results_b):
    """
    Perform a single permutation test on two result vectors.
    """
    vector_length = len(results_a)
    swap_vector = np.random.randint(0, 2, vector_length)
    permuted_a = results_a.copy()
    permuted_b = results_b.copy()
    for i in range(vector_length):
        if swap_vector[i] == 1:
            permuted_a[i] = results_b[i]
            permuted_b[i] = results_a[i]
    original_difference = abs(np.mean(results_a) - np.mean(results_b))
    permuted_difference = abs(np.mean(permuted_a) - np.mean(permuted_b))
    return permuted_difference >= original_difference


def permutation_test(results_a, results_b):
    """
    Perform a single permutation test on two result vectors. It returns the probability that the null hypothesis is true
    """
    s = 0
    for i in range(R):
        if single_permute(results_a, results_b):
            s += 1
    p = (s + 1) / (R + 1)
    return p

In [0]:
if PERMUTATION:
    permutation_scores = {}
    for model_name in recall_arrays.keys():
        permutation_scores[model_name] = {}

    for model_name1, pred1 in recall_arrays.items():
        for model_name2, pred2 in recall_arrays.items():
            permutation_scores[model_name1][model_name2] = permutation_test(np.array(pred1), np.array(pred2))

    pp.pprint(permutation_scores)


In [0]:
def get_doc_text(id):
    cursor = conn.cursor()
    cursor.execute(
        "SELECT lines FROM documents WHERE id = ?",
        (id,)
    )
    result = cursor.fetchone()
    cursor.close()
    return result

def parse_doc(doc_raw):
    """
    Parse a list of lines from a raw document text, with the index in the list
    correponding to the line index in the data entries
    """
    new = []
    lines = doc_raw.split("\n")
    char_count = 0
    for line in lines:
        # print('Line: {}'.format(line))
        line = line.split("\t")
        
    #   TODO: THIS MIGHT DROP PARTS OF SENTENCES AFTER A TAB
        if len(line[1]) > 1:
            new.append(line[1])
            char_count += len(line[1])
        else:
            new.append(EMPTY_TOKEN)
    chars.append(char_count)
    return new

In [0]:
fever_db = 'fever.db'
root = '/content/drive/My Drive/Overig/'

db = os.path.join(root, fever_db)
conn = sqlite3.connect(db)
EMPTY_TOKEN = 'E'
chars = []

In [0]:
def get_sentence(doc, idx):
    # print(f"Get doc {doc} index {idx}")
    doc_raw = get_doc_text(doc)[0]
    # print(doc_raw)
    parsed_doc = parse_doc(doc_raw)
    sentence = parsed_doc[idx]
    return sentence

In [0]:
random.Random(4).shuffle(errors)
selected_errors = errors[:100]
for e in selected_errors:
    sentence = get_sentence(e[2], e[3])
    e.append(sentence)
    predictions = e[5]
    e.remove(predictions)
    for i in range(5):
        if i < len(predictions):
            e.append(get_sentence(predictions[i][0], predictions[i][1]))
        else:
            e.append(None)


# pp.pprint(selected_errors)

In [0]:
selected_errors_pd = pd.DataFrame(selected_errors, columns=['claim_id', 'claim_text', 'doc_id', 'sentence_index', 'label', 'sentence_text', 'wrong1', 'wrong2', 'wrong3', 'wrong4', 'wrong5'])
selected_errors_pd.to_csv('/content/drive/My Drive/Overig/error_analysis.csv')

In [0]:
for e in errors:
    label = 1 if e[4] == 'SUPPORTS' else 0
    e.append(label)
errors_pd = pd.DataFrame(errors, columns=['claim_id', 'claim_text', 'doc_id', 'sentence_index', 'label', 'label_int'])
np.mean(errors_pd.label_int)